In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
!ls /data2/yt8m/code/yt8m

data_converter.py  probs.npy			rnn3_finetune_val_0919_5319.npy
loader.py	   probs_rnn3_tr_0919_1943.npy	settings.py
models.py	   __pycache__			train.py
notebooks	   radam.py			utils.py
probs_100.npy	   rnn3_0918.npy		work_dirs


In [3]:
# p = np.load('/data2/yt8m/code/yt8m/probs_rnn3_tr_0919_1943.npy', allow_pickle=True) lb0.5xx

In [4]:
p1 = np.load('/data2/yt8m/code/yt8m/rnn3_finetune_val_0919_5319.npy', allow_pickle=True)
p1[:2]

array([[7.6846051e-01, 1.2664025e-05, 7.3119008e-06, ..., 9.0855399e-05,
        1.5706182e-05, 5.4914362e-05],
       [8.4863472e-01, 3.3775061e-05, 7.3738171e-05, ..., 1.0154960e-05,
        1.4125281e-03, 3.9235205e-05]], dtype=float32)

In [5]:
p2 = np.load('/data2/yt8m/code/yt8m/probs.npy', allow_pickle=True)
p2[:2]

array([[6.9659978e-01, 1.3740591e-04, 1.1694086e-06, ..., 8.6689353e-05,
        6.0337617e-05, 2.5136296e-07],
       [8.1797791e-01, 4.7672194e-07, 1.8004628e-05, ..., 1.0984526e-08,
        5.2207708e-04, 1.4610782e-07]], dtype=float32)

In [6]:
p3 = np.load('/data2/yt8m/code/yt8m/rnn3_0918.npy', allow_pickle=True)
p3[:2]

array([[8.8766348e-01, 7.4459531e-05, 1.4046773e-07, ..., 1.0157839e-05,
        3.3870427e-04, 8.4951431e-08],
       [9.9903846e-01, 4.3215941e-11, 9.2645269e-09, ..., 5.1636928e-12,
        5.8856360e-05, 4.3446281e-11]], dtype=float32)

In [7]:
p = np.average([p1, p2, p3], 0, weights=[0.7, 0.15, 0.15])

In [8]:
type(p)

numpy.ndarray

In [9]:
p[:2]

array([[7.75561848e-01, 4.06446333e-05, 5.31481204e-06, ...,
        7.81258584e-05, 7.08506102e-05, 3.84905008e-05],
       [8.66596758e-01, 2.37140573e-05, 5.43188033e-05, ...,
        7.11012071e-06, 1.07590969e-03, 2.74865661e-05]])

In [10]:
df_test = pd.read_csv('/data2/yt8m/3/frame/test_ids.csv')
df_test.head()

,vid,start_time,seg_id
0,MeVj,0,MeVj:0
1,MeVj,5,MeVj:5
2,MeVj,10,MeVj:10
3,MeVj,15,MeVj:15
4,MeVj,20,MeVj:20


In [11]:
def get_classes():
    df = pd.read_csv('/data2/yt8m/3/frame/classes_1001.csv')
    c = df['class'].values.tolist()
    stoi = { c[i]: i for i in range(len(c)) }
    return c, stoi

classes, _ = get_classes()

In [12]:
classes[:2]

['none', '3']

In [13]:
print(df_test.shape, p.shape)

(2038114, 3) (2038114, 1001)


In [14]:
res = []
for i in tqdm(range(1, 1001)):
    idx = np.argsort(p[:,i])[::-1][:100000]
    best_segs = df_test.seg_id.values[idx]
    pred_str = ' '.join(best_segs)
    res.append({
        'Class': classes[i],
        'Segments': pred_str
    })


100%|██████████| 1000/1000 [05:32<00:00,  3.01it/s]


In [15]:
df_sub = pd.DataFrame(res)
df_sub.head()

,Class,Segments
0,3,1shy:15 LXlb:65 1shy:150 Rbop:5 1shy:100 1shy:...
1,7,WEoW:105 WEoW:85 NQHs:35 WEoW:115 hSim:135 3F8...
2,8,sYqm:40 sYqm:190 sYqm:90 sYqm:145 sYqm:70 sYqm...
3,11,j6wU:220 2JGm:175 2JGm:135 bQG6:255 uue7:5 2JG...
4,12,odzE:265 L7U7:15 JJ9a:210 FXxf:100 FXxf:90 FXx...


In [16]:
df_sub.to_csv('ens_0919_1.csv', index=False)

In [17]:
!kaggle competitions submit -c youtube8m-2019 -f ens_0919_1.csv -m 'submit'

100%|████████████████████████████████████████| 812M/812M [00:23<00:00, 36.8MB/s]
Successfully submitted to The 3rd YouTube-8M Video Understanding Challenge

In [18]:
!ls -lh

total 4.9G
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 5.8M Sep 18 14:09 create_meta.ipynb
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  41K Sep 18 15:56 eda.ipynb
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 812M Sep 19 06:41 ens_0919_1.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 810M Sep 17 19:27 sub1_rnn3_local057_100000.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  82M Sep 17 19:16 sub1_rnn3_local057_10000.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users  17M Sep 17 19:06 sub1_rnn3_local057_2000.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 8.2M Sep 17 03:34 sub1_rnn3_local0.57.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 810M Sep 18 04:53 sub_ens1.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 9.1K Sep 19 06:40 submit.ipynb
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 812M Sep 18 03:29 sub_rnn3_att_local060_100000.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.domain users 814M Sep 19 03:27 sub_rnn3_tr_0919_1943.csv
-rw-r--r-- 1 FAREAST.chec FAREAST.d

In [19]:
del res

In [20]:
del p